Import Libraries

In [100]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

Load Dataset

Dataset Link = https://www.kaggle.com/code/alpertemel/fer2013-with-keras/input

In [101]:
data = pd.read_csv("fer2013.csv")

Select the Specific Labels

In [102]:
INTERESTED_LABELS = [3, 6]

Load the Dataset of above specific labels

In [103]:
df = data[data.emotion.isin(INTERESTED_LABELS)]
df.shape

(15187, 3)

Data Preprocessing

In [104]:
# Extract features and labels

X = df['pixels'].apply(lambda x: np.array([int(pixel) for pixel in x.split()])).values
y = df['emotion'].values

# Split the dataset

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train

array([array([225, 234, 194, ..., 227, 226, 224]),
       array([59, 25, 22, ..., 79, 83, 84]),
       array([ 72,  98, 106, ..., 209, 231, 113]), ...,
       array([255, 255, 254, ..., 255, 248, 254]),
       array([ 78,  77,  80, ..., 219, 203, 185]),
       array([  2,   2,   3, ..., 116, 118, 120])], dtype=object)

In [105]:
# Flatten the images for SVM

X_train_flat = np.array([image.flatten() for image in X_train])
X_test_flat = np.array([image.flatten() for image in X_test])
X_train_flat

array([[225, 234, 194, ..., 227, 226, 224],
       [ 59,  25,  22, ...,  79,  83,  84],
       [ 72,  98, 106, ..., 209, 231, 113],
       ...,
       [255, 255, 254, ..., 255, 248, 254],
       [ 78,  77,  80, ..., 219, 203, 185],
       [  2,   2,   3, ..., 116, 118, 120]])

In [106]:
# Normalize pixel values to be between 0 and 1

X_train_flat = X_train_flat / 255.0
X_test_flat = X_test_flat / 255.0
X_train_flat

array([[0.88235294, 0.91764706, 0.76078431, ..., 0.89019608, 0.88627451,
        0.87843137],
       [0.23137255, 0.09803922, 0.08627451, ..., 0.30980392, 0.3254902 ,
        0.32941176],
       [0.28235294, 0.38431373, 0.41568627, ..., 0.81960784, 0.90588235,
        0.44313725],
       ...,
       [1.        , 1.        , 0.99607843, ..., 1.        , 0.97254902,
        0.99607843],
       [0.30588235, 0.30196078, 0.31372549, ..., 0.85882353, 0.79607843,
        0.7254902 ],
       [0.00784314, 0.00784314, 0.01176471, ..., 0.45490196, 0.4627451 ,
        0.47058824]])

Dimensionality Reduction

In [107]:
# Perform PCA for dimensionality reduction

pca = PCA(n_components=50)  # Adjust the number of components as needed
X_train_pca = pca.fit_transform(X_train_flat)
X_test_pca = pca.transform(X_test_flat)
X_train_pca

array([[-2.17210889e-01,  7.72077382e+00,  9.02949832e-01, ...,
         5.12507713e-03, -2.25504688e-01, -7.20479433e-01],
       [ 7.93580341e+00, -1.37188058e-01,  6.77943688e-02, ...,
         2.22430167e-01, -1.05178409e+00,  4.50752148e-01],
       [-2.63007174e+00,  2.65341063e+00,  2.99695647e+00, ...,
         4.61586750e-01, -9.01020068e-02,  4.41914905e-01],
       ...,
       [-8.00607638e+00,  3.57697914e+00,  7.14260809e-02, ...,
         2.75369079e-01,  3.77409297e-01,  1.06194160e+00],
       [-1.10425319e+00, -7.68229497e-01,  1.58445221e+00, ...,
         2.42903877e-01, -4.90718163e-02, -3.12839682e-01],
       [ 4.30422549e+00, -7.62025339e+00, -1.19804794e+00, ...,
        -2.90078492e-01,  1.24497790e-01, -3.17206292e-01]])

Standardize

In [108]:
# Standardize features

scaler = StandardScaler()
X_train_pca_std = scaler.fit_transform(X_train_pca)
X_test_pca_std = scaler.transform(X_test_pca)
X_train_pca_std

array([[-0.03506044,  1.99732977,  0.25437277, ...,  0.01036465,
        -0.46078244, -1.48134221],
       [ 1.28093378, -0.03548994,  0.01909856, ...,  0.44982957,
        -2.14915104,  0.92676925],
       [-0.4245251 ,  0.6864255 ,  0.84428182, ...,  0.93348566,
        -0.18410891,  0.90859943],
       ...,
       [-1.29227667,  0.92534856,  0.02012166, ...,  0.55689009,
         0.77117499,  2.18340573],
       [-0.1782397 , -0.19873755,  0.4463609 , ...,  0.49123439,
        -0.10027034, -0.64321423],
       [ 0.69475358, -1.97132558, -0.33750577, ..., -0.58663753,
         0.25439114, -0.65219221]])

Model Training and Evaluation - SVM

In [109]:
# Train SVM Model

svm_model = SVC(kernel='linear', C=1)
svm_model.fit(X_train_pca, y_train)

SVC(C=1, kernel='linear')

In [110]:
# Evaluate Models

svm_predictions = svm_model.predict(X_test_pca)
svm_accuracy = accuracy_score(y_test, svm_predictions)
print("SVM Accuracy:", svm_accuracy)
print("SVM Classification Report:\n", classification_report(y_test, svm_predictions))
svm_classification_report = classification_report(y_test, svm_predictions,output_dict=True)

SVM Accuracy: 0.7027649769585254
SVM Classification Report:
               precision    recall  f1-score   support

           3       0.72      0.81      0.76      1784
           6       0.67      0.55      0.60      1254

    accuracy                           0.70      3038
   macro avg       0.69      0.68      0.68      3038
weighted avg       0.70      0.70      0.70      3038



In [111]:
# Evaluate SVM Model for each label separately

svm_predictions = svm_model.predict(X_test_pca)
for label in INTERESTED_LABELS:
    label_indices = np.where(y_test == label)[0]
    label_predictions = svm_predictions[label_indices]
    label_true = y_test[label_indices]
    label_accuracy = accuracy_score(label_true, label_predictions)
    print(f"SVM Accuracy for Label {label}:", label_accuracy)

SVM Accuracy for Label 3: 0.8099775784753364
SVM Accuracy for Label 6: 0.5502392344497608


Model Training and Evaluation - Naïve Bayes

In [112]:
# Train Naïve Bayes Model

nb_model = GaussianNB()
nb_model.fit(X_train_flat, y_train)

GaussianNB()

In [113]:
nb_predictions = nb_model.predict(X_test_flat)
nb_accuracy = accuracy_score(y_test, nb_predictions)
print("Naïve Bayes Accuracy:", nb_accuracy)
print("Naïve Bayes Classification Report:\n", classification_report(y_test, nb_predictions))

Naïve Bayes Accuracy: 0.5826201448321264
Naïve Bayes Classification Report:
               precision    recall  f1-score   support

           3       0.65      0.63      0.64      1784
           6       0.49      0.52      0.51      1254

    accuracy                           0.58      3038
   macro avg       0.57      0.57      0.57      3038
weighted avg       0.59      0.58      0.58      3038



In [114]:
# Evaluate Naïve Bayes Model for each label separately

nb_predictions = nb_model.predict(X_test_flat)
for label in INTERESTED_LABELS:
    label_indices = np.where(y_test == label)[0]
    label_predictions = nb_predictions[label_indices]
    label_true = y_test[label_indices]
    label_accuracy = accuracy_score(label_true, label_predictions)
    print(f"Naïve Bayes Accuracy for Label {label}:", label_accuracy)

Naïve Bayes Accuracy for Label 3: 0.6261210762331838
Naïve Bayes Accuracy for Label 6: 0.5207336523125997
